In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import pandas as pd

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

from utils.general import make_sure_path_exists

In [ ]:
# exp. data
setsizes = [9, 16, 25, 36]
subjects = np.arange(49)

stimulus_dir = 'data/stimuli/'
stimuli = [s for s in os.listdir(stimulus_dir)]

output_dir = 'results/exemplar_visual-search-trajectories/'
make_sure_path_exists(output_dir)

In [ ]:
def make_fix_screen(r=20):
    """
    make fixation cross screen
    """
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    # add fix cross
    fix_draw = ImageDraw.Draw(screen) 
    screen_center = np.array([1280, 1024])/2
    # horizontal
    fix_draw.line((screen_center[0]-r, screen_center[1],
                   screen_center[0]+r, screen_center[1]),
                  fill='white', width=5)
    # vertical
    fix_draw.line((screen_center[0], screen_center[1]-r,
                   screen_center[0], screen_center[1]+r),
                  fill='white', width=5)
    return screen

In [ ]:
def make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir):
    """
    make choice set screen
    """
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    for i, stimfile in enumerate(trial_stimuli):
        # get stim pos
        stimbox = stimulus_positions.loc[i].values.astype(np.int)
        # get target size
        stimsize = (stimbox[2]-stimbox[0], stimbox[3]-stimbox[1])
        # load stim img
        stimimg = Image.open(stimulus_dir+stimfile)
        # resize stim
        stimimg = stimimg.resize(size=stimsize)
        # paste
        screen.paste(stimimg, box=stimbox)
    return screen

In [ ]:
def draw_gaze_marker(screen, pos, color='white', width=5):
    """
    add current gaze position to screen
    """
    # init draw
    box_draw = ImageDraw.Draw(screen)
    # horizontal left
    box_draw.line((pos[0], pos[1],
                   pos[2], pos[1]),
                  fill=color, width=width)
    # horizontal right
    box_draw.line((pos[0], pos[3],
                   pos[2], pos[3]),
                  fill=color, width=width)
    # vertical top
    box_draw.line((pos[0], pos[1],
                   pos[0], pos[3]),
                  fill=color, width=width)
    # vertical bottom
    box_draw.line((pos[2], pos[1],
                   pos[2], pos[3]),
                  fill=color, width=width)
    return screen

In [ ]:
# generate visual search GIF for each empirical trial

np.random.seed(1)

for setsize in setsizes:
    # load data
    setsize_data = pd.read_csv('data/summary_files/{}_data.csv'.format(setsize))
    stimulus_positions = pd.read_csv('data/stimulus_positions/{}_stimulus_positions.csv'.format(setsize), header=None)
    stimulus_centers = np.concatenate(((stimulus_positions.loc[:,0].values+0.5*(stimulus_positions.loc[:,2].values-stimulus_positions.loc[:,0].values))[:,None],
                                       (stimulus_positions.loc[:,1].values+0.5*(stimulus_positions.loc[:,3].values-stimulus_positions.loc[:,1].values))[:,None]), axis=1)
    for subject in subjects:
        # subset & load data
        subject_data = setsize_data[setsize_data['subject']==subject].copy()
        subject_gaze_data = pd.read_csv('data/subject_files/{}_{}_fixations.csv'.format(subject, setsize))
        # get trials
        trials = subject_gaze_data.trial.unique()
        for trial in trials:
            gif_path = output_dir+'setsize-{}_sub-{}_trial-{}.gif'.format(setsize, subject, trial)
            avi_path = output_dir+'setsize-{}_sub-{}_trial-{}.avi'.format(setsize, subject, trial)
            if not os.path.isfile(gif_path) or not os.path.isfile(avi_path):
                print('Processing: setsize: {}, subject: {}, trial: {}'.format(setsize, subject, trial))
                # subset data
                trial_data = subject_data[subject_data['trial']==trial].copy()
                trial_gaze_data = subject_gaze_data[subject_gaze_data['trial']==trial].copy()
                # get trial stimuli
                trial_stimuli = trial_data[['stimulus_{}'.format(i) for i in range(setsize)]].values[0]
                # make output dir
                trial_path = output_dir+'sub-{}_settsize-{}_trial-{}/'.format(subject, setsize, trial)
                # make fix screen
                fix_screen = make_fix_screen()
                # make choice screen
                trial_img = [fix_screen]
                for fi, fixitem in enumerate(trial_gaze_data['item'].values.astype(np.int)):
                    fix_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                    # add fixation markers
                    fix_screen = draw_gaze_marker(fix_screen, stimulus_positions.loc[fixitem])
                    trial_img.append(fix_screen)
                # add choice indication
                choice_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                choice_screen = draw_gaze_marker(choice_screen,
                                                 stimulus_positions.loc[trial_data['choice'].values[0]],
                                                 color='red', width=5)
                trial_img.append(choice_screen)
                # make gif & save
                trial_img[0].save(gif_path, save_all=True, optimize=False, append_images=trial_img[1:], loop=0, 
                                  duration=[500*2]+list(trial_gaze_data['dur'].values.astype(np.int)*2)+[500*2])